In [25]:
#Load model from file
import pandas as pd
import numpy as np

def loadmodelfromfile():
    f = open(modelname+'.txt', 'r')
    f.readline()
    f.readline()
    fx = []
    A = []
    B = []
    p = []
    l = '_'
    while 1:
        l = f.readline().rstrip()
        if l != '': 
            fx.append(l) 
        else: 
            break
    while 1:
        l = f.readline().rstrip()
        if l != '':
            if l.__contains__(','):
                l = l.split(',')
                l = [float(x) for x in l]
                A.append(l)
            else:    
                A.append(float(l))
        else:
            break
    while 1:
        l = f.readline().rstrip()
        if l != '':
            if l.__contains__(','):
                l = l.split(',')
                l = [float(x) for x in l]
                B.append(l)
            else:    
                B.append(float(l))
        else:
            break
    f.readline()
    f.readline()
    for x in f.readline().rstrip().split(','):
        p.append(int(x))
    f.readline()
    e = float(f.readline().rstrip())

    print("Model: %s" % modelname)
    print("Function list: ")
    print(fx) 
    print("A matrice: ")
    print(A)
    print("B matrice: ")
    print(B)
    print("Priority factor array (most important to least important): ")
    print(p)
    print("Limit for the ratio between the weights ranked first and last: ")
    print(e)

    return fx, A, B, p, e

In [35]:
#Conversion functions
import numpy as np
import math

def lin(lin_y,lin_X,lin_Y):
    lin_m = (lin_Y[1]-lin_Y[0])/(lin_X[1]-lin_X[0])
    lin_b = lin_Y[0] - lin_m*lin_X[0]
    lin_x = (lin_y - lin_b)/lin_m
    return lin_x

def converttovaluescale(el,uf_i,uf_m):
    match uf_m:
        case 'lin':
            index = np.absolute(np.array(A[uf_i])-el).argmin()
            A_uf_i = np.array(A[uf_i])
            B_uf_i = np.array(B[uf_i])
            return (lin(el,B_uf_i[index:index+2],A_uf_i[index:index+2])) 
        case 'exp':
            return A[uf_i]*math.exp(el)+B[uf_i]
        case 'log':
            return A[uf_i]*math.log(el)+B[uf_i]
        case _:
            return -999


#Convert data to value scale
def convertoriginaltovaluescale():
    #Read data
    df1 = pd.read_csv(modelname+'_originals.csv')
    #print(df1.head(10))

    #Convert
    n = len(fx)
    for i in range(1,n+1):
        df1.iloc[:, i] = df1.iloc[:, i].apply(lambda x: converttovaluescale(x,i-1,fx[i-1]))
    #print(df1.head(10))

    df1
    #Save data
    df1_headers = ['DMUs']
    for i in range(1,n+1):
        df1_headers.append('V'+str(i))
    df1.columns = df1_headers
    df1.to_csv(modelname+"_valuescale.csv",float_format='%.4f', index=False)

In [45]:
#Phase one with no restrictions
import numpy as np
import pandas as pd
from scipy.optimize import linprog

def phase1():
    #Load values
    df2 = pd.read_csv(modelname+'_valuescale.csv')
    df2 = df2.iloc[:,1:]
    nw = len(df2.columns)
    ndmu = len(df2)
    #Load into numpy array
    #_A = df2.to_numpy()
    #print(_A)

    _A = df2.to_numpy()
    print(_A)

    #Run Solver
    res_fase1 = []
    for i in range(ndmu):
        A2 = np.copy(_A)
        A_27 = np.copy(_A[i])
        A2[i] = np.zeros(nw)
        A3 = A2 - A_27.T
        A_ub = np.c_[A3,-1*np.ones((ndmu,1))]
        c = np.array([0,0,0,0,0,1])
        b_ub= np.zeros(ndmu)
        A_eq = [np.array([1,1,1,1,1,0])]
        b_eq = [1]
        wn_bounds = [(0,None),(0,None),(0,None),(0,None),(0,None),(None,None)]
        res = linprog(c, A_ub=A_ub, b_ub=b_ub,A_eq=A_eq, b_eq=b_eq,bounds=wn_bounds)
        #print('Optimal value:', res.fun, '\nX:', res.x)
        res2 = np.zeros(nw+2)
        res2[0] = i+1
        #print(res2)
        #print(res.x)
        res2[1:] = res.x
        print(res2)
        res_fase1.append(res2)
    print(res_fase1)
    #Save data to file
    column_names = []
    column_names.append('DMUs')
    for i in range(1,nw+1):
        column_names.append('w'+str(i))
    column_names.append('d*')    
    df3 = pd.DataFrame(data = res_fase1, columns=column_names)
    print(df3.head(10))
    df3.to_csv(modelname+"_phase11.csv", float_format='%.4f', index = False)

#Run model
modelname = 'Model2'

phase1()

[[0.6326 0.8237 0.572  0.7435 0.1234]
 [0.5293 0.7889 0.7883 0.7335 0.1016]
 [0.5266 0.8242 0.4747 0.7851 0.0702]
 [0.7464 0.7354 0.8247 0.6866 0.1028]
 [0.8204 0.8088 0.7527 0.708  0.051 ]
 [0.411  0.4063 0.4133 0.5518 0.2476]
 [0.0838 0.1153 0.173  0.3    0.5847]
 [0.3547 0.4503 0.2201 0.1361 0.3338]
 [0.6936 0.7871 0.7854 0.9449 0.1009]
 [0.4161 0.318  0.5992 0.5782 0.2265]
 [0.2015 0.2526 0.3264 0.2857 0.3969]
 [0.35   0.3469 0.2194 0.4518 0.3458]]
[ 1.00000000e+00  7.35974305e-03  4.67597831e-01  3.22100028e-11
  3.80525911e-11  5.25042426e-01 -2.84785909e-02]
[ 2.00000000e+00  4.06521270e-11  4.31868154e-01  5.68131845e-01
  1.06658747e-11  4.85940482e-10 -2.42494503e-03]
[ 3.00000000e+00  1.18438111e-11  8.46218487e-01  2.74366543e-11
  1.53781513e-01  1.65629182e-11 -6.82042016e-03]
[ 4.00000000e+00  2.05015666e-01  2.83339558e-09  7.94984331e-01
  1.14421809e-10  6.15665703e-10 -4.20677112e-02]
[ 5.00000000e+00  9.99999999e-01  3.22811511e-10  1.43074826e-10
  8.65222897e-10  

In [23]:
#Phase two with no restrictions
import numpy as np
import pandas as pd
from scipy.optimize import linprog

modelname = 'Model2'

df4 = pd.read_csv(modelname+'_phase1.csv')
print(df4.head())

df4 = df4.iloc[:,1:]
nw = len(df4.columns)
ndmu = len(df4)

   DMUs            U1            U2            U3            U4            U5  \
0   1.0  7.171798e-03  4.670932e-01  4.169168e-11  4.678374e-11  5.257350e-01   
1   2.0  3.846693e-11  4.312579e-01  5.687421e-01  1.023693e-11  4.870360e-10   
2   3.0  1.188794e-11  8.463198e-01  2.759340e-11  1.536802e-01  1.664150e-11   
3   4.0  2.052838e-01  2.780805e-09  7.947162e-01  1.057737e-10  6.018995e-10   
4   5.0  1.000000e+00  3.447010e-10  1.404840e-10  8.454798e-10  1.833974e-10   

          d  
0 -0.028467  
1 -0.002393  
2 -0.006866  
3 -0.042024  
4 -0.073999  


In [17]:
#Convert to original scales
import numpy as np
import pandas as pd
import math

def lin(lin_y,lin_X,lin_Y):
    lin_m = (lin_Y[1]-lin_Y[0])/(lin_X[1]-lin_X[0])
    lin_b = lin_Y[0] - lin_m*lin_X[0]
    lin_x = (lin_y - lin_b)/lin_m
    return lin_x

def converttooriginalscale(el,uf_i,uf_m):
    match uf_m:
        case 'lin':
            #print(uf_i)
            #print(A[uf_i])
            #index = np.absolute(np.array(A[uf_i])-el).argmin()
            #A_uf_i = np.array(A[uf_i])
            #B_uf_i = np.array(B[uf_i])
            #return (lin(el,B_uf_i[index:index+2],A_uf_i[index:index+2]))
            return 2 
        case 'exp':
            return math.log((el-B[uf_i])/A[uf_i])
        case 'log':
            return math.exp((el-B[uf_i])/A[uf_i])
        case _:
            return -999


#Convert data to value scale
def convertvaluescaletooriginal():
    #Read data
    df = pd.read_csv(modelname+'_valuescale.csv')
    df = df.iloc[:,1:]
    #Convert
    n = len(fx)
    for i in range(1,n+1):
        df.iloc[:, i] = df.iloc[:, i].apply(lambda x: converttooriginalscale(x,i-1,fx[i-1]))
    #print(df.head(10))

    #Save data
    df_headers = ['DMUs']
    for i in range(1,n):
        df_headers.append('X'+str(i))
    #Cuidado pode haver mais outputs
    df.append('Y')
    df.columns = df_headers
    df.to_csv(modelname+"_final.csv",index=False)

convertvaluescaletooriginal()

         V1        V2        V3        V4        V5
0  0.632606  0.823685  0.572013  0.743496  0.123417
1  0.529306  0.788853  0.788322  0.733450  0.101625
2  0.526573  0.824244  0.474699  0.785096  0.070219
3  0.746379  0.735360  0.824677  0.686588  0.102844
4  0.820378  0.808836  0.752683  0.707954  0.050969
5  0.411009  0.406275  0.413348  0.551850  0.247583
6  0.083775  0.115270  0.172962  0.299986  0.584750
7  0.354696  0.450283  0.220133  0.136075  0.333792
8  0.693562  0.787105  0.785441  0.944943  0.100937
9  0.416121  0.317980  0.599236  0.578170  0.226500
5
0
log
1
log
2
log
3
lin
4
lin


IndexError: single positional indexer is out-of-bounds

In [ ]:
#Convert model from original to utility scale
modelname = 'Model2'

#Convert model from 
fx = []
A = []
B = []
p = []
fx,A,B,p,e = loadmodelfromfile()
convertoriginaltovaluescale()

In [38]:
#Run model
modelname = 'Model2'

phase1()



[[0.6326 0.8237 0.572  0.7435 0.1234]
 [0.5293 0.7889 0.7883 0.7335 0.1016]
 [0.5266 0.8242 0.4747 0.7851 0.0702]
 [0.7464 0.7354 0.8247 0.6866 0.1028]
 [0.8204 0.8088 0.7527 0.708  0.051 ]
 [0.411  0.4063 0.4133 0.5518 0.2476]
 [0.0838 0.1153 0.173  0.3    0.5847]
 [0.3547 0.4503 0.2201 0.1361 0.3338]
 [0.6936 0.7871 0.7854 0.9449 0.1009]
 [0.4161 0.318  0.5992 0.5782 0.2265]
 [0.2015 0.2526 0.3264 0.2857 0.3969]
 [0.35   0.3469 0.2194 0.4518 0.3458]]
[ 1.00000000e+00  7.35974305e-03  4.67597831e-01  3.22100028e-11
  3.80525911e-11  5.25042426e-01 -2.84785909e-02]
[ 2.00000000e+00  4.06521270e-11  4.31868154e-01  5.68131845e-01
  1.06658747e-11  4.85940482e-10 -2.42494503e-03]
[ 3.00000000e+00  1.18438111e-11  8.46218487e-01  2.74366543e-11
  1.53781513e-01  1.65629182e-11 -6.82042016e-03]
[ 4.00000000e+00  2.05015666e-01  2.83339558e-09  7.94984331e-01
  1.14421809e-10  6.15665703e-10 -4.20677112e-02]
[ 5.00000000e+00  9.99999999e-01  3.22811511e-10  1.43074826e-10
  8.65222897e-10  